In [12]:
import numpy as np

def binomial_option_pricing(S0, K, T, r, sigma, n, option_type='call'):

    dt = T / n  # Length of each time step
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability
    
    stock_prices = np.zeros(n + 1)
    option_values = np.zeros(n + 1)
    
    for i in range(n + 1):
        stock_prices[i] = S0 * (u ** i) * (d ** (n - i))
    
    if option_type == 'call':
        option_values = np.maximum(stock_prices - K, 0)
    elif option_type == 'put':
        option_values = np.maximum(K - stock_prices, 0)
    
    for j in range(n - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i + 1] + (1 - p) * option_values[i])
    
    return option_values[0]

def calculate_confidence_interval(S0, K, T, r, sigma, n, option_type='call', num_simulations=1000):
    prices = np.array([
        binomial_option_pricing(S0, K, T, r, np.random.normal(sigma, 0.05), n, option_type) 
        for _ in range(num_simulations)
    ])
    
    mean_price = np.mean(prices)
    std_dev = np.std(prices)
    
    z_score = 1.96  # For a 95% confidence level
    margin_of_error = z_score * (std_dev / np.sqrt(num_simulations))
    
    lower_ci = mean_price - margin_of_error
    upper_ci = mean_price + margin_of_error
    
    return mean_price, lower_ci, upper_ci

# Example parameters
S0 = 450.0       # Initial stock price of Nvidia
K = 460.0        # Strike price
T = 1.0          # Time to maturity (1 year)
r = 0.05         # Risk-free interest rate (5% per annum)
sigma = 0.30     # Volatility (30% per annum)
n = 100          # Number of steps in the binomial tree
option_type = 'call'  # Type of option ('call' or 'put')
num_simulations = 1000  # Number of simulations to run

# Calculate the option price and confidence interval
mean_price, lower_ci, upper_ci = calculate_confidence_interval(S0, K, T, r, sigma, n, option_type, num_simulations)
print(f"The estimated price of the {option_type} option is: ${mean_price:.2f}")
print(f"95% confidence interval: (${lower_ci:.2f}, ${upper_ci:.2f})")


The estimated price of the call option is: $59.91
95% confidence interval: ($59.38, $60.44)
